<a href="https://colab.research.google.com/github/Shreeranga1/KNNClassifier/blob/main/KNNPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('/content/sample_data/bank.csv')

In [3]:
df.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,32,205,2,999,0,1.1,93.994,-36.4,4.858,5191.0,no
1,32,691,10,999,0,1.4,93.918,-42.7,4.960,5228.1,yes
2,45,45,8,999,0,1.4,93.444,-36.1,4.963,5228.1,no
3,33,400,1,5,2,-1.1,94.601,-49.5,1.032,4963.6,yes
4,47,903,2,999,1,-1.8,93.075,-47.1,1.415,5099.1,yes


In [5]:
Q1=df.quantile(0.25)
Q3=df.quantile(0.75)
IQR=Q3-Q1

In [6]:
first_quartile=Q1-1.5*IQR

In [7]:
third_quartile=Q3+1.5*IQR

In [8]:
df=df[~((df< first_quartile)|(df>third_quartile)).any(axis=1)]

In [9]:
df=df.reset_index(drop=True)

In [10]:
df.shape

(6555, 11)

Split the independent and dependent variable

In [11]:
df_target=df['y']
df_feature=df.drop('y',axis=1)

In [12]:
df_target.replace({'yes':1,'no':0},inplace=True)

In [16]:
df_target.astype('int')

0       0
1       1
2       0
3       0
4       0
       ..
6550    1
6551    0
6552    0
6553    0
6554    0
Name: y, Length: 6555, dtype: int64

Do null check

In [18]:
Total=df_feature.isnull().sum()
Total

age               0
duration          0
campaign          0
pdays             0
previous          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
dtype: int64

In [19]:
df_feature.std()

age                10.184894
duration          253.406706
campaign            1.283539
pdays               0.000000
previous            0.000000
emp.var.rate        1.626927
cons.price.idx      0.585132
cons.conf.idx       4.659679
euribor3m           1.779666
nr.employed        74.431765
dtype: float64

In [20]:
df_feature.drop(labels=["pdays","previous"],axis=1,inplace=True)

SCale the data

In [21]:
from sklearn.preprocessing import StandardScaler

In [22]:
s=StandardScaler()
df_scaled=s.fit_transform(df_feature)
X=pd.DataFrame(df_scaled,columns=df_feature.columns)
X.head()

,age,duration,campaign,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,-0.722787,-0.480765,-0.024486,0.701361,0.749298,0.910658,0.765585,0.394074
1,-1.410132,-0.330797,0.754670,-1.081276,-1.217928,-2.008221,-1.104009,-0.840708
2,-0.330018,-0.445246,-0.803642,-0.036282,-0.607764,-0.291233,0.350869,0.458567
3,0.357326,0.880787,-0.803642,0.701361,0.749298,0.910658,0.765023,0.394074
4,1.241055,-1.037225,-0.803642,0.701361,0.749298,0.910658,0.763899,0.394074


Now lets split the data 

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train,X_test,Y_train,Y_test=train_test_split(X,df_target,random_state=10,test_size =0.2)

KNN implementation

In [25]:
from sklearn.neighbors import KNeighborsClassifier

In [28]:
Knn=KNeighborsClassifier(n_neighbors=3,metric='euclidean')
Knn_model=Knn.fit(X_train,Y_train)
y_pred=Knn.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score

In [31]:
accuracy_score(Y_test,y_pred)

0.8497330282227308

In [32]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(Y_test,y_pred)

array([[715, 104],
       [ 93, 399]])

In [35]:
from sklearn.metrics import classification_report

In [37]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88       819
           1       0.79      0.81      0.80       492

    accuracy                           0.85      1311
   macro avg       0.84      0.84      0.84      1311
weighted avg       0.85      0.85      0.85      1311



In [38]:
from sklearn.metrics import roc_auc_score

In [39]:
roc_auc_score(Y_test,y_pred)

0.8419957413859853

Tuned Params

In [41]:
from sklearn.model_selection import GridSearchCV

In [47]:
tuned_paramaters = {'n_neighbors': np.arange(1, 25, 2),
                   'metric': ['hamming','euclidean','manhattan','Chebyshev']}
knn_classification = KNeighborsClassifier(n_jobs=-1)
knn_grid = GridSearchCV(estimator = knn_classification, 
                        param_grid = tuned_paramaters, 
                        cv = 3, 
                        scoring = 'accuracy', 
                        n_jobs=-1)

In [48]:
knn_grid.fit(X_train, Y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'metric': ['hamming', 'euclidean', 'manhattan',
                                    'Chebyshev'],
                         'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [50]:
knn_grid.best_params_

{'metric': 'manhattan', 'n_neighbors': 15}

In [54]:
knn_classification = KNeighborsClassifier(n_neighbors = knn_grid.best_params_['n_neighbors'], 
                                          metric=knn_grid.best_params_['metric'])

# fit the model using fit() on train data
knn_model = knn_classification.fit(X_train, Y_train)

In [55]:
y_pred=knn_model.predict(X_test)
accuracy_score(Y_test,y_pred)

0.858886346300534